# 3. Fake News - Classification Model Tuning
In this project we're going to quickly load a dataset called Real_or_Fake_News and run a through a simple classification modelling process and then tune some basic huperparameters. 

The data consists of news headlines and articles which we explored in part 1. In this case we are training and comparing results from two models (MultinomialNB and LogisticRegression) on the text column. 

Unlike part 2. we are using GridSearch, Cross_val and a custom scorer for a more thorough model validation in this execise.

In [23]:
#Quick inspection of the data
import pandas as pd
pd.set_option('display.max_colwidth', 100) 


df = pd.read_csv('fake_or_real_news.csv')

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB
None


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focu...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Political Suicide At A Trump Rally (VIDEO),Google Pinterest Digg Linkedin Reddit Stumbleupon Print Delicious Pocket Tumblr \nThere are two ...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,"U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, am...",REAL
3,10142,Bernie supporters on Twitter erupt in anger against the DNC: 'We tried to warn you!',"— Kaydee King (@KaydeeKing) November 9, 2016 The lesson from tonight's Dem losses: Time for Demo...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners Hillary Clinton and Donald Trump are leading in t...,REAL


In [24]:
# Encoding the labels
df['label'] = df['label'].replace('FAKE', 1)
df['label'] = df['label'].replace('REAL', 0)

In [ ]:
# Setting fetures X and labels y
X = df['text'] # +' '+ df['text'] 
y = df['label']

# Import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize two TfidfVectorizers object: basic and tuned 
tuned_tfidf = TfidfVectorizer(stop_words='english', max_df=0.5,
                                   max_features=25000, ngram_range=(2,2))

# Transform the data 
tfidf_fit = tuned_tfidf.fit_transform(X)

# Create the DataFrames
X = pd.DataFrame(tfidf_fit.toarray(), columns=tuned_tfidf.get_feature_names())

In [ ]:
print(X.shape)
print(y.shape)

### Time to model

In [ ]:
import numpy as np
import seaborn as sns
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, make_scorer, accuracy_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler

In [ ]:
# Step 2 use a dictionary of key(model name):value(instantiated)pairs to instantiate the models
def get_models():
    # Empty dict to 
    models = {}
    
    models['MNB'] = MultinomialNB()
    models['LGR'] = LogisticRegression()
    models['SVC'] = SVC()
    return models  

In [17]:
models = get_models()

### Custom scorer returns accuracy_score and also stores labels as arrays to load into a classification_report

In [21]:
def custom_scoring(y, y_pred):
    originalclass.extend(y)
    preditedclass.extend(y_pred)
    return accuracy_score(y, y_pred)

In [22]:
%%time
originalclass = []
preditedclass = []http://localhost:8888/notebooks/Documents/GitHub/Personal-Projects/Fake%20News%20Classifier%20%26%20WordCloud%20/3.%20Fake%20News%20-%20Classification%20Model%20Tuning.ipynb#
results = []
names = []

for name, model in models.items():
    # Loading a 3 split KFold into cross val
    cv = KFold(n_splits=3)
    scores = cross_val_score(model, X, y, scoring= make_scorer(custom_scoring), cv=cv, n_jobs =1, error_score='raise')
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))

sns.boxplot(x=names, y=results, showmeans=True)

SyntaxError: invalid syntax (<unknown>, line 2)

In [20]:
len(originalclass)

NameError: name 'originalclass' is not defined

In [114]:
print(classification_report(originalclass[:6336], preditedclass[:6336]))
print(classification_report(originalclass[6335:12671], preditedclass[6335:12671]))

              precision    recall  f1-score   support

           0       0.90      0.94      0.92      3171
           1       0.94      0.89      0.92      3165

    accuracy                           0.92      6336
   macro avg       0.92      0.92      0.92      6336
weighted avg       0.92      0.92      0.92      6336

              precision    recall  f1-score   support

           0       0.95      0.88      0.91      3171
           1       0.89      0.95      0.92      3164

    accuracy                           0.92      6335
   macro avg       0.92      0.92      0.92      6335
weighted avg       0.92      0.92      0.92      6335



### Let's dig a little deeper with GridSearchCV and Cross Val

In [123]:
from sklearn.model_selection import GridSearchCV

MNB_param_grid = {'alpha': np.arange(0.1,1,0.1)}

LGR_param_grid = {'max_iter': [200],
                  'solver': ['lbfgs', 'liblinear'],
                   'C': np.arange(0.1, 1, 0.2)}

In [121]:
%%time

gridsearch = GridSearchCV(models['MNB'], MNB_param_grid, n_jobs=1, scoring='f1')
gridsearch.fit(X, y) 
cv_scores = cross_val_score(gridsearch, X, y)
print(cv_scores)
print(gridsearch.best_estimator_)

[0.93108974 0.91983806 0.91800643 0.91368078 0.90805533]
MultinomialNB(alpha=0.2)
CPU times: user 5min 30s, sys: 1min 33s, total: 7min 3s
Wall time: 5min 48s


In [124]:
%%time

gridsearch = GridSearchCV(models['LGR'], LGR_param_grid, n_jobs=1, scoring='f1')
gridsearch.fit(X, y) 
cv_scores = cross_val_score(gridsearch, X, y)
print(cv_scores)
print(gridsearch.best_estimator_)

[0.92645944 0.91527988 0.92940264 0.92785105 0.9164751 ]
LogisticRegression(C=0.9000000000000001, max_iter=200, solver='liblinear')
CPU times: user 16min 42s, sys: 1min 48s, total: 18min 31s
Wall time: 12min 37s


In [118]:
# gridsearch.cv_results_['split0_test_score']